In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))

In [57]:
from utils import *
from plots import *

In [58]:
df_test = pd.read_csv('../data/cleaned_test_data.csv')

In [59]:
df = pd.read_csv('../data/train.csv')

/tmp/ipykernel_87515/3253700651.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/train.csv')


In [60]:
column_summary(df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Store,int64,0,1017209,1115,"{1115: 942, 1: 942, 2: 942, 3: 942, 4: 942, 5:..."
1,DayOfWeek,int64,0,1017209,7,"{5: 145845, 4: 145845, 3: 145665, 2: 145664, 1..."
2,Date,object,0,1017209,942,"{'2013-01-02': 1115, '2015-07-31': 1115, '2015..."
3,Sales,int64,0,1017209,21734,"{0: 172871, 5674: 215, 5558: 197, 5483: 196, 6..."
4,Customers,int64,0,1017209,4086,"{0: 172869, 560: 2414, 576: 2363, 603: 2337, 5..."
5,Open,int64,0,1017209,2,"{1: 844392, 0: 172817}"
6,Promo,int64,0,1017209,2,"{0: 629129, 1: 388080}"
7,StateHoliday,object,0,1017209,5,"{'0': 855087, 0: 131072, 'a': 20260, 'b': 6690..."
8,SchoolHoliday,int64,0,1017209,2,"{0: 835488, 1: 181721}"


In [61]:
df.StateHoliday.unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [62]:
df_test['StateHoliday'] = df_test['StateHoliday'].map({'0': 0, 'a': 1, 'b': 2, 'c': 3})

In [63]:
df['StateHoliday'] = df['StateHoliday'].map({'0': 0, 'a': 1, 'b': 2, 'c': 3})


In [64]:
df_test['Date'] = pd.to_datetime(df_test['Date'])

df_test['Year'] = df_test['Date'].dt.year
df_test['Month'] = df_test['Date'].dt.month
df_test['Day'] = df_test['Date'].dt.day

In [65]:
df['Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [66]:
df_test['Is_Weekend'] = df_test['Date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

In [67]:
df['Is_Weekend'] = df['Date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

In [68]:
# Identify dates with holidays (StateHoliday != 0)
holiday_mask = df_test['StateHoliday'] != 0
holiday_dates = df_test.loc[holiday_mask, 'Date']

# If there are no holidays, the model will break, so ensure there's at least one holiday
if len(holiday_dates) == 0:
    raise ValueError("No holidays found in the dataset!")
# Append the first and last holiday to the beginning and end of the list, to simulate cyclic behavior
holiday_dates = pd.concat([holiday_dates, holiday_dates + pd.DateOffset(years=1)])


# Calculate "Number of days to holiday" (loop back to the next year's holidays if needed)
df_test['Days_to_Holiday'] = df_test['Date'].apply(lambda x: (holiday_dates[holiday_dates >= x] - x).min().days)

# Calculate "Number of days after holiday" (loop back to the previous year's holidays if needed)
df_test['Days_after_Holiday'] = df_test['Date'].apply(lambda x: (x - holiday_dates[holiday_dates <= x]).min().days)

# Display the DataFrame with the new columns
print(df_test[['Date', 'StateHoliday', 'Days_to_Holiday', 'Days_after_Holiday']])

            Date  StateHoliday  Days_to_Holiday  Days_after_Holiday
0     2015-09-17             0              333                33.0
1     2015-09-17             0              333                33.0
2     2015-09-17             0              333                33.0
3     2015-09-17             0              333                33.0
4     2015-09-17             0              333                33.0
...          ...           ...              ...                 ...
41083 2015-08-01             0               14                 NaN
41084 2015-08-01             0               14                 NaN
41085 2015-08-01             0               14                 NaN
41086 2015-08-01             0               14                 NaN
41087 2015-08-01             0               14                 NaN

[41088 rows x 4 columns]


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
# Identify dates with holidays (StateHoliday != 0)
holiday_mask = df['StateHoliday'] != 0
holiday_dates = df.loc[holiday_mask, 'Date']

# If there are no holidays, the model will break, so ensure there's at least one holiday
if len(holiday_dates) == 0:
    raise ValueError("No holidays found in the dataset!")

# Append the first and last holiday to the beginning and end of the list, to simulate cyclic behavior
holiday_dates = pd.concat([holiday_dates, holiday_dates + pd.DateOffset(years=1)])

# Calculate "Number of days to holiday" (loop back to the next year's holidays if needed)
df['Days_to_Holiday'] = df['Date'].apply(lambda x: (holiday_dates[holiday_dates >= x] - x).min().days)

# Calculate "Number of days after holiday" (loop back to the previous year's holidays if needed)
df['Days_after_Holiday'] = df['Date'].apply(lambda x: (x - holiday_dates[holiday_dates <= x]).min().days)

# Display the DataFrame with the new columns
print(df[['Date', 'StateHoliday', 'Days_to_Holiday', 'Days_after_Holiday']])

In [72]:
def categorize_day(day):    
    if day >= 1 and day <= 10:
        return 0  # Beginning of the Month
    elif day >= 11 and day <= 20:
        return 1  # Mid-Month
    else:
        return 2  # End of the Month


In [73]:
# Apply the function to create a new column
df_test['MonthPhase'] = df_test['Day'].apply(categorize_day)

In [74]:
# Apply the function to create a new column
df['MonthPhase'] = df['Day'].apply(categorize_day)

In [75]:
df_test.fillna(-1, inplace=True)

In [ ]:
save_dataframe_to_csv(df_test, 'final_test.csv')

In [ ]:
save_dataframe_to_csv(df, 'train_final.csv')

In [78]:
column_summary(df)

,col_name,col_dtype,num_of_nulls,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Store,int64,0,1017209,1115,"{1115: 942, 1: 942, 2: 942, 3: 942, 4: 942, 5:..."
1,DayOfWeek,int64,0,1017209,7,"{5: 145845, 4: 145845, 3: 145665, 2: 145664, 1..."
2,Date,datetime64[ns],0,1017209,942,"{2013-01-02 00:00:00: 1115, 2015-07-31 00:00:0..."
3,Sales,int64,0,1017209,21734,"{0: 172871, 5674: 215, 5558: 197, 5483: 196, 6..."
4,Customers,int64,0,1017209,4086,"{0: 172869, 560: 2414, 576: 2363, 603: 2337, 5..."
5,Open,int64,0,1017209,2,"{1: 844392, 0: 172817}"
6,Promo,int64,0,1017209,2,"{0: 629129, 1: 388080}"
7,StateHoliday,float64,131072,886137,4,"{0.0: 855087, 1.0: 20260, 2.0: 6690, 3.0: 4100}"
8,SchoolHoliday,int64,0,1017209,2,"{0: 835488, 1: 181721}"
9,Year,int32,0,1017209,3,"{2013: 406974, 2014: 373855, 2015: 236380}"
